### Task 1

#### Import required libraries

In [3]:
import json
import pandas as pd
import urllib.request
import urllib
import requests
from datetime import date as dt
import psycopg2
from sqlalchemy import create_engine

#### Import the JSON file

In [4]:
path = './Downloads/data-analyst-challenge-master/data-analyst-challenge-master/orders.json' # To be updated before execution
f = open(path,)
data = json.load(f)

In [5]:
data

[{'id': 8369263756632563,
  'customer': {'id': 47178,
   'name': 'William Doe',
   'email': 'william.doe@gmail.com'},
  'total_price': 79.39,
  'created_at': '2020-03-07T14:31:11Z',
  'line_items': [{'id': 610448,
    'product_id': 632910392,
    'product_sku': 'PPLEPUNCH20-05',
    'product_name': 'Purple Punch 2.0 Dried Flower',
    'price': 29.4},
   {'id': 997208,
    'product_id': 278266679,
    'product_sku': 'GPSTASH-01',
    'product_name': "Grandpa's Stash Dried Flower",
    'price': 49.99}]},
 {'id': 7262170348080494,
  'customer': {'id': 94720,
   'name': 'Emile Tumson',
   'email': 'emile.tumson@gmail.com'},
  'total_price': 86.98,
  'created_at': '2020-03-01T09:16:30Z',
  'line_items': [{'id': 997208,
    'product_id': 278266679,
    'product_sku': 'GPSTASH-01',
    'product_name': "Grandpa's Stash Dried Flower",
    'price': 49.99},
   {'id': 759658,
    'product_id': 573794972,
    'product_sku': 'SAGENSOUR-01',
    'product_name': 'Sage N Sour Dried Flower',
    'price'

#### Parse the JSON data and store in three dataframes - customer data, order data and order line items

In [6]:
# The below lists will be combined into a dataframe
# List to store the order IDs
orderID = []
# List to store the total price per order
totalPrice = []
# List to store the order dates 
orderDate = []
# List to store the customer Ids
customerId = []
# Create empty dataframe to store order line items 
orderItem = pd.DataFrame()
# Create empty dataframe to store customer data
customer = pd.DataFrame()
# Loop through each item in json object
for d in data:
    orderID.append(d['id'])
    totalPrice.append(d['total_price'])
    orderDate.append(d['created_at'].split('T')[0])
    customerId.append(d['customer']['id'])
    # Create a list with the order id to be appended with the order line items
    j = [d['id'] for i in range(len(d['line_items']))]
    # Dataframe with the lien items for a specific order with order id as the index
    tempOrder = pd.DataFrame(d['line_items'], index = j)
    orderItem = pd.concat([orderItem, tempOrder])
    # Dataframe to store customer data
    customer = pd.concat([customer, pd.DataFrame(d['customer'], index = [d['id'],])])
    

In [7]:
# Combine the lists for OrderId, total price, order date and customer id to form a dataframe for orders. Here customer id acts as a foreign key
orders = pd.DataFrame(list(zip(orderID, totalPrice, orderDate, customerId)), 
               columns =['orderID', 'totalPrice', 'orderDate', 'customerId']) 
# Use index as column OrderId in OrderItem dataframe 
orderItem['orderId'] = orderItem.index

# Reset the index in customer dataframe
customer = customer.reset_index(drop=True)

#### Get currency exchange rate from API

In [8]:
# Get the currency exchange for a span of 1 year starting from Aug 1, 2019 to Aug 1, 2020. Base currency is USD and get exchange rate for CAD
base = 'https://api.exchangeratesapi.io/'
currency = '&base=USD&symbols=CAD'

URL = base + 'history?start_at=' + '2019-08-01' + '&end_at=' + '2020-08-01' + currency
response = requests.get(URL)
currency_json = response.json()

In [9]:
# Create dataframe of the exchange rates for the 1 year duration
date = []
rate = []
for key in currency_json['rates']:
    date.append(key)
    rate.append(currency_json['rates'][key]['CAD'])

exchangeRate = pd.DataFrame(list(zip(date, rate)), 
               columns =['date', 'rate']) 

In [10]:
# Calculate average rate over the 1 year duration. This value will be used to fill missing rates for the orders that
# were placed on weekends/holidays when the market was closed 
rateAvg = exchangeRate['rate'].mean()
rateAvg

1.345653248287109

In [11]:
# Merge the exchange rate with orders dataframe on orderdate to get a column for 'rate' in the orders table 
orders = pd.merge(orders, exchangeRate, left_on = 'orderDate', right_on = 'date', how = 'left')
orders['rate'].fillna(rateAvg, inplace=True)

In [12]:
orders

,orderID,totalPrice,orderDate,customerId,date,rate
0,8369263756632563,79.39,2020-03-07,47178,NaN,1.345653
1,7262170348080494,86.98,2020-03-01,94720,NaN,1.345653
2,7722055557038194,36.99,2020-01-01,59933,NaN,1.345653
3,4280852170163518,55.98,2020-03-10,23955,2020-03-10,1.367779
4,5058732129957127,309.98,2020-03-15,59933,NaN,1.345653
5,6431399815998774,315.97,2020-03-12,23955,2020-03-12,1.381139
6,5791957346866373,36.99,2020-03-24,47178,2020-03-24,1.449414
7,2079640804901496,55.98,2020-02-19,23955,2020-02-19,1.322315
8,2249838395290890,309.98,2020-03-13,47178,2020-03-13,1.385897
9,7638742701884072,35.39,2020-03-08,59933,NaN,1.345653


In [13]:
customer

,id,name,email
0,47178,William Doe,william.doe@gmail.com
1,94720,Emile Tumson,emile.tumson@gmail.com
2,59933,Ethan Jones,ethan.jones@gmail.com
3,23955,Daniel Smith,daniel.smith@gmail.com
4,59933,Ethan Jones,ethan.jones@gmail.com
5,23955,Daniel Smith,daniel.smith@gmail.com
6,47178,William Doe,william.doe@gmail.com
7,23955,Daniel Smith,daniel.smith@gmail.com
8,47178,William Doe,william.doe@gmail.com
9,59933,Ethan Jones,ethan.jones@gmail.com


In [14]:
orderItem

,id,product_id,product_sku,product_name,price,orderId
8369263756632563,610448,632910392,PPLEPUNCH20-05,Purple Punch 2.0 Dried Flower,29.40,8369263756632563
8369263756632563,997208,278266679,GPSTASH-01,Grandpa's Stash Dried Flower,49.99,8369263756632563
7262170348080494,997208,278266679,GPSTASH-01,Grandpa's Stash Dried Flower,49.99,7262170348080494
7262170348080494,759658,573794972,SAGENSOUR-01,Sage N Sour Dried Flower,36.99,7262170348080494
7722055557038194,759658,573794972,SAGENSOUR-01,Sage N Sour Dried Flower,36.99,7722055557038194
4280852170163518,317808,477365777,GOGPR-07,Glueberry OG Pre-Rolls,5.99,4280852170163518
4280852170163518,771854,694165761,HYBTINC-03,Hybrid Blend THC Tincture,49.99,4280852170163518
5058732129957127,604665,510650809,PAX-3,Pax 3 Vaporizer,259.99,5058732129957127
5058732129957127,997208,278266679,GPSTASH-01,Grandpa's Stash Dried Flower,49.99,5058732129957127
6431399815998774,997208,278266679,GPSTASH-01,Grandpa's Stash Dried Flower,49.99,6431399815998774


### Task 2

The approach I have used below doees not require creating a schema of the table in the database. The table is created at the same time when the data is uploaded. However, the script for creating tables in SQl is as below.

CREATE TABLE public.customer
(
    id bigint,
    name text,
    email text
)

CREATE TABLE public.orders
(
    orderId bigint,
    totalPrice int,
    orderDate date,
    customerId bigint,
    rate float
)

CREATE TABLE public.orderItem
(
    orderId bigint,
    id bigint,
    productName text,
    itemPrice float,
    productId bigint,
    productSku text
)

#### Upload data into PostgreSQL

In [15]:
# Reusable function to insert data into a table
# Reference: https://pythontic.com/pandas/serialization/postgresql
def insertTable(conn, table, df):
    postgreSQLTable = table;
    try:
        frame = df.to_sql(postgreSQLTable, postgreSQLConnection, if_exists='fail');
    except ValueError as vx:
        print('ValueError', vx)
        
    except Exception as ex:  
        print('exception',ex)
    else:
        print("PostgreSQL Table %s has been created successfully."%postgreSQLTable);

In [17]:
username = 'postgres'
password = 'sweetshe09'
dbname = 'namasteTech'
alchemyEngine = create_engine('postgresql+psycopg2://'+username+':' + password + '@127.0.0.1/' + dbname, pool_recycle=3600);
postgreSQLConnection = alchemyEngine.connect();
insertTable(postgreSQLConnection, 'orders', orders)
insertTable(postgreSQLConnection, 'customer', customer)
insertTable(postgreSQLConnection, 'orderItem', orderItem)
postgreSQLConnection.close();

PostgreSQL Table orders has been created successfully.
PostgreSQL Table customer has been created successfully.
PostgreSQL Table orderItem has been created successfully.


### Task 3

Link to the public dashboard
https://public.tableau.com/profile/suguna.menon#!/vizhome/NamasteTechnologyAssessment/Ordercountpercustomer?publish=yes